In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random

driver = webdriver.Chrome()
driver.get("https://www.zerozero.pt/edition.php?id_edicao=156405")

In [11]:
def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [12]:
page_source = driver.page_source

In [13]:
soup = BeautifulSoup(page_source, 'html.parser')


In [14]:
html_scorers = soup.find("div", class_="rbbox nofooter")
children = html_scorers.findChildren("td" , recursive=True)
# print(b.prettify())

scorers = {}
top_assist = {}
assists = []
values = []
for x in range(1,len(children)):
    child = children[x].find("div", class_="text")
    if child != None:
        # print(child.contents[0].text)
        assists.append(child.contents[0].text)
        # scorers[child.contents[0].text] = 
        # scorers[child]
for x in range(1,len(children)):
    child = children[x].find("div", class_="ball_numbers")
    if child != None:
        # print(child.contents[0].text)
        values.append(child.contents[0].text)

        # scorers[child.contents[0].text] = 
        # scorers[child]
# print(scorers)

for x in range(len(assists)-2):
    scorers[assists[x]] = values[x]

top_assist[assists[len(assists)-1]] = values[-1]

html_league_image = soup.find("div",class_="profile_picture")
league_logo = "https://www.zerozero.pt/" + html_league_image.find("img").get("src")
print(league_logo)

https://www.zerozero.pt/img/logos/edicoes/156405_imgbank_.png


In [15]:
html_resume = soup.find_all("div", class_="edition_resume_stats")
stats = {}

stats["goals/game"] = html_resume[0].text
stats["victories_home"] = html_resume[1].text
stats["Ties"] = html_resume[2].text
stats["victories_away"] = html_resume[3].text
stats["logo"] = league_logo
print(stats)


{'goals/game': '2,50', 'victories_home': '43%', 'Ties': '19%', 'victories_away': '38%', 'logo': 'https://www.zerozero.pt/img/logos/edicoes/156405_imgbank_.png'}


In [16]:
b = soup.find(id="DataTables_Table_0").findChildren("tbody")
# print(b[1])

clubs = {}

for x in b[0]:
    # print(x.prettify())
    club_name = x.find("a")
    values = x.find_all("td")
    clubs[club_name.text] = [val.text for val in values]
    clubs[club_name.text] += [club_name['href']]
    time.sleep(1)
    club_page = driver.get("https://www.zerozero.pt"+club_name['href'])
    soup2 = BeautifulSoup(driver.page_source, 'html.parser')
    team_html = soup2.find_all(id="team_squad")

    #get link to club image
    html_club_image = soup2.find("div",class_="profile_picture")
    club_logo = "https://www.zerozero.pt/" + html_club_image.find("img").get("src")

    #get resume of the competitions it plays
    html_competitions = soup2.find(id="entity_season")
    lines = html_competitions.find_all("td")
    y = list(divide_chunks(lines, 6))
    competitions = {}
    for i in range(len(y)-1):
        competitions[y[i][0].text] = [y[i][1].text, y[i][2].text, y[i][3].text, y[i][4].text, y[i][5].text]

    # #get the titles the club has
    # html_titles = soup2.find(id="coach_titles")
    # titles = {}
    # entries = html_titles.find_all("div", class_="trophy")
    # for entry in entries:
    #     titles[entry.find("div", class_="text").text] = entry.find("div", class_="number").text

    #get the club info
    html_club_bio = soup2.find_all(id="entity_bio")
    bio = html_club_bio[0].find_all("div", class_="bio")
    bio_half = html_club_bio[0].find_all("div", class_="bio_half")
    # print(bio)
    club_info =  []
    for y in bio:

        if y.contents[1].text == "UEFA: ":
            club_info.append(y.contents[2].text)
            if len(y.contents) > 4:
                club_info.append(y.contents[5].text)
        else:
            club_info.append(y.contents[1].text)


    for y in bio_half:
        club_info.append([y.contents[1].text])

    #get team players
    team_players = {}
    for team in team_html[0]:
        players_per_position = team.find_all("div", class_="staff")
        for player in players_per_position:
            number = player.find("div", class_="number").text
            player_name = player.find("div", class_="name").find("div", class_="text").text
            age = player.find("div", class_="name").find("span").text
            link = player.find("div", class_="name").find("div", class_="text").find("a")["href"]
            time.sleep(1)
            player_page = driver.get("https://www.zerozero.pt/"+link)
            # time.sleep(random.randint(1, 5))


            soup3 = BeautifulSoup(driver.page_source, 'html.parser')

            html_team_games = soup3.find(id="team_games")

            if html_team_games != None:
                lines = html_team_games.find_all("td")

                l = list(divide_chunks(lines, 5))
                competitions_player = {}
                for i in range(len(l)-1):
                    competitions_player[l[i][0].text] = [l[i][1].text, l[i][2].text, l[i][3].text, l[i][4].text]

            html_player_titles = soup3.find(id="coach_titles")
            player_titles = {}
            if html_player_titles != None:
                entries_player = html_player_titles.find_all("div", class_="trophy")
                for y in entries_player:
                    player_titles[y.find("div", class_="competition").text] = y.find("div", class_="number").text

            html_history_player = soup3.find(id="coach_career").find("table")
            player_clubs = {}
            entries = html_history_player.find_all("td")
            for i in range(3):
                player_clubs[entries[6*i+1].text] = [entries[6*i+2].text,entries[6*i+3].text, entries[6*i+4].text, entries[6*i+5].text]


            #get link to player image
            html_player_image = soup3.find("div",class_="profile_picture")
            player_logo = html_player_image.find("img").get("src")


            a = soup3.find_all(id="entity_bio")
            children = a[0].findChildren("div" , recursive=False)
            dateBirth = countryBirth =  nationality = secondNationality = naturality = position = preferedFoot = height = weight = ""
            for x in range(1,len(children)):
                child = children[x]
                if child.find("span").text =="Nome":
                    name = child.contents[1]
                    # print(name)
                elif child.contents[0].text =="Nascimento":
                    dateBirth = child.contents[1]
                    # print(dateBirth)
                    if child.find("div", class_="text") == "None":
                        if child.findChildren("span", recursive=False)[2].text == "País de Nascimento":
                            countryBirth = child.find("div", class_="text").text
                        # print(countryBirth)
                elif child.findChildren("span", recursive=False)[0].text == "Nacionalidade":
                    nationality = child.contents[1].contents[1].text
                    # print(nationality)
                    if len(child.contents) > 3:
                        if child.contents[2].text == "Dupla Nacionalidade":
                            secondNationality = children[3].contents[3].contents[1].text
                            # print(secondNationality)
                elif child.find("span").contents[0] == "Naturalidade":
                    naturality = child.contents[1]
                    # print(naturality)
                elif child.find("span").contents[0] == "Posição":
                    position = child.contents[1].text
                    # print(position)
                elif child.find("span").contents[0] == "Pé preferencial":
                    if len(child.contents) > 1:
                        preferedFoot = child.contents[1].text
                        # print(preferedFoot)
                elif child.find("span").contents[0] == "Altura":
                    height = child.contents[1].text
                    # print(height)
                elif child.find("span").contents[0] == "Peso":
                    weight = child.contents[1].text
                    # print(weight)

            # history = soup3.find_all("select", class_="chosen-select-search")
            # player_clubs = []
            # # print(history[4])
            # for x in history[4].find_all("option"):
            #     player_clubs.append([x.text, x.attrs["value"]])


            player[name] = [dateBirth, countryBirth, nationality, secondNationality, naturality, position, preferedFoot, height, weight, "https://www.zerozero.pt/"+link]
            # player[name] += [player_clubs]
            team_players[number] = [name, age, dateBirth, countryBirth, nationality, secondNationality, naturality, position, preferedFoot, height, weight, player_logo] + [competitions_player, player_titles, player_clubs]

    clubs[club_name.text] += [team_players, competitions, club_info, club_logo, "https://www.zerozero.pt"+club_name['href']]
    # print(clubs)
    # print(name.text)
    # print("HELLO")
    # break


# print(clubs)

In [44]:
print(len(clubs))

18


In [9]:
print(titles)

{'II Liga Portuguesa': '4', 'III Divisão': '1'}


In [ ]:
print(clubs)


In [ ]:
# print(soup3.prettify())
history = soup3.find_all("select", class_="chosen-select-search")
for x in history[4].find_all("option"):
    print(x.text)
    print(x.attrs["value"])

In [17]:
import json

with open('22_23.txt', 'w') as convert_file:
     convert_file.write(json.dumps(clubs))


with open('22_23_stats.txt', 'w') as convert_file:
     convert_file.write(json.dumps([stats,scorers,top_assist]))

In [ ]:
# player_clubs = {}
# html_player = soup3.find_all("tr")
# # print(html_player)

# # teams = a[0].findChildren("tbody")
# print(len(b))
# for x in b:
#     print(x)
#     children = b[0].findChildren("td" , recursive=False)
#     season = child.contents[1]
#     print(len(season))

    
# children = a[0].find("table",class_="career")
# # table = children.find_all("tr")
# print((children))
